In [2]:
"""環境"""
# .lab

"""更新履歴"""
# 最終更新日：2025/05/21
# 2025/05/21 shutilを使って、cachedirを洗浄することにしました
# 2025/05/15 simple_progress_barおよびdata(dict)を使い始めました。
# 2025/02/25 os.getcwd()を採用し、__file__を定義する必要をなくしました。

"""モジュール読み込み"""
# ファイル操作等
import sys
import os
# from datetime import datetime
from pprint import pprint
import logging
# import pickle
# import struct
from tqdm import tqdm
import h5py
# import threading
import json

# tkinter
from tkinter import filedialog, messagebox, Tk

# データ分析ツール
import pandas as pd
import numpy as np
import scipy as sp
# import math
# from sklearn.linear_model import LinearRegression

# グラフ等作成用
import matplotlib
import matplotlib.pyplot as plt         # 図の作成用
from PIL import Image as im
# import cv2
from IPython.display import display, HTML, clear_output, update_display, Image

# 自作モジュール
sys.path.append(r"C:\Users\okaza\pythonenv")
from modules.Mytools.Tools import print_fileinfo, h5_tree, dict_tree, simple_progress_bar, clean_cache_except_logfiles, get_total_size
import modules.Mytools.Settings
import modules.fitXRD as fx
from modules.peakfit import peakfit, pseudoVoigt

"""ログ管理ツール作成"""
# chche directoryの設定
cachedir = os.path.abspath(os.getcwd() + "/.cache")
clean_cache_except_logfiles(cachedir)
os.makedirs(cachedir, exist_ok=True)

# loggerの作成
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = "%(levelname)-9s  %(asctime)s [%(filename)s:%(lineno)d] %(message)s"

# Streamハンドラクラスを作成
sh = logging.StreamHandler()
sh.setLevel(logging.DEBUG)
sh.setFormatter(logging.Formatter(format))
logger.addHandler(sh)

# Fileハンドラクラスをインスタンス化
logfile = cachedir + ""
fh = logging.FileHandler(filename=cachedir + "/notebook.log", encoding="utf-8")
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter(format))
logger.addHandler(fh)
logger.debug("[Activate workspace]: " + os.getcwd())
logger.debug("[Set cache]: "+ os.path.abspath(cachedir))
logger.debug("[Activate log]: " + os.path.abspath(cachedir + "/notebook.log"))

# globalなデータを作成
data = dict()
logger.debug("[Create global variable]: data")

DEBUG      2025-06-27 17:06:36,459 [388102471.py:71] [Activate workspace]: c:\Users\okaza\pythonenv\lf\anim_rawdata
DEBUG      2025-06-27 17:06:36,461 [388102471.py:72] [Set cache]: c:\Users\okaza\pythonenv\lf\anim_rawdata\.cache
DEBUG      2025-06-27 17:06:36,463 [388102471.py:73] [Activate log]: c:\Users\okaza\pythonenv\lf\anim_rawdata\.cache\notebook.log
DEBUG      2025-06-27 17:06:36,464 [388102471.py:77] [Create global variable]: data


## 目的

生データをアニメーションでまとめます。

## 1. データ読み込み

speファイルを読み込みます。

In [3]:
from modules.SpeReadPy.read_spe import SpeReference

In [4]:
def read_spe():

    # spe生データ
    spefile = r"C:\Users\okaza\Box\DataStrorage\SPring8-2024-10\Okazaki\mDIH05_0028\mDAC_03  29.spe"

    if True: # [ Main ]
        # データ読み込み
        spe = SpeReference(spefile)
        logger.info("[File]: " + os.path.abspath(spefile))

        # データ格納
        key = sys._getframe().f_code.co_name
        data[key] = dict()
        data[key]["data"] = spe.get_data()[0]
        data[key]["wavelength"] = spe.get_wavelengths()[0]
        logger.info("[Add variables]: {} (data = {:.2f} MB)".format(key, float(get_total_size(data))/1024/1024))
        dict_tree(data)

    return
read_spe()
del read_spe

INFO       2025-06-27 17:06:45,416 [3739853959.py:9] [File]: C:\Users\okaza\Box\DataStrorage\SPring8-2024-10\Okazaki\mDIH05_0028\mDAC_03  29.spe
INFO       2025-06-27 17:06:46,127 [3739853959.py:16] [Add variables]: read_spe (data = 363.96 MB)



└── read_spe
    ├── data <class 'numpy.ndarray'> [shape = (89, 400, 1340)]
    └── wavelength <class 'numpy.ndarray'> [shape = (1340,)]

## 2. 動画作成

In [5]:
from matplotlib import animation as anim

In [8]:
def save_animation():
    
    # データ読み込み
    intensities = data["read_spe"]["data"]
    wavelength = data["read_spe"]["wavelength"]
    n_frame = intensities.shape[0]
    wavelengthlim = (wavelength[0], wavelength[-1])

    # fig作成
    fig, ax = plt.subplots()
    fig.set_size_inches((6,4.5))
    
    # 上軸の作成
    wax = ax.twiny()
    wax.set_xlim(wavelengthlim)
    wax.set_xlabel("Wavelength [nm]", fontsize = 10)

    # アニメーションの作成
    ims = []
    for i_frame in range(n_frame):
        imshow = ax.imshow(
            data["read_spe"]["data"][i_frame],
            aspect = "auto",
            cmap="gray"
        )
        text = ax.text(
            0,
            1.1,
            "frame = {:>4}".format(i_frame),
            transform = ax.transAxes,
            fontsize = 10
        )
        ims.append([imshow, text])
        simple_progress_bar(i_frame+1, n_frame)

    # 保存
    ani = anim.ArtistAnimation(fig, ims, interval = 300)
    logger.debug("[Save]: Animation")
    key = sys._getframe().f_code.co_name
    filename = cachedir + "/{}.mp4".format(key)
    ani.save(filename, writer = "ffmpeg", fps = 10, dpi = 300)
    logger.info("[Save]: " + os.path.abspath(filename))
    plt.close()

    return
save_animation()
del save_animation

Progress: [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■----------]  76% (68/89) 

DEBUG      2025-06-27 17:42:31,538 [4086732273.py:38] [Save]: Animation


Progress: [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100% (89/89) 


INFO       2025-06-27 17:43:06,678 [4086732273.py:42] [Save]: c:\Users\okaza\pythonenv\lf\anim_rawdata\.cache\save_animation.mp4
